## 1. Feature Engineering Objective

The goal of this notebook is to transform order-level food delivery data into
customer-level behavioral features that can be used for churn prediction.

Key principles:
- Avoid data leakage
- Use only historical information
- Engineer business-meaningful features
- Prepare a model-ready dataset

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

In [2]:
DATA_PATH = Path("../data/raw")
df = pd.read_csv(DATA_PATH / "order_history.csv")

In [3]:
# Keep only delivered orders (EDA decision)
df = df[df["Order Status"] == "Delivered"].copy()

# Convert date column
df["Order Placed At"] = pd.to_datetime(df["Order Placed At"], errors="coerce")

/var/folders/r5/39ntwzvn2wl22mr5517j8m6h0000gn/T/ipykernel_5784/807149095.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Order Placed At"] = pd.to_datetime(df["Order Placed At"], errors="coerce")


In [4]:
## 3. Drop High-Sparsity Columns
drop_cols = [
    "Order Ready Marked",
    "Restaurant penalty (Rejection)",
    "Restaurant compensation (Cancellation)",
    "Cancellation / Rejection reason",
    "Review",
    "Customer complaint tag",
    "Instructions",
    "Rating",
    "Discount construct"
]

df = df.drop(columns=drop_cols, errors="ignore")

In [6]:
## 4. Missing Value Treatment
df["KPT duration (minutes)"] = df["KPT duration (minutes)"].fillna(
    df["KPT duration (minutes)"].median()
)

df["Rider wait time (minutes)"] = df["Rider wait time (minutes)"].fillna(
    df["Rider wait time (minutes)"].median()
)

## Define Reference Date

All features are computed relative to a fixed reference date
to prevent data leakage.

In [7]:
reference_date = df["Order Placed At"].max()
reference_date

Timestamp('2025-01-31 23:59:00')

In [9]:
## Core Customer Aggregation
customer_features = df.groupby("Customer ID").agg(
    total_orders=("Order ID", "count"),
    avg_order_value=("Total", "mean"),
    total_spend=("Total", "sum"),
    last_order_date=("Order Placed At", "max"),
    first_order_date=("Order Placed At", "min")
).reset_index()
customer_features

,Customer ID,total_orders,avg_order_value,total_spend,last_order_date,first_order_date
0,000285ae83ecf06a92b936d4f5b74342edb0e1940e1f00...,3,453.6000,1360.80,2024-10-01 21:32:00,2024-09-03 21:41:00
1,001ab5fc3ee158b4d22e106897cee9b355e6eed50f163e...,1,1332.4000,1332.40,2025-01-10 01:38:00,2025-01-10 01:38:00
2,001fdf2511dd137361424c9c15ea54774476691ceec5fa...,1,1352.4000,1352.40,2024-12-13 22:26:00,2024-12-13 22:26:00
3,002afbd83626f0c699892fcb07b2ddf3858911482647b6...,4,805.6225,3222.49,2024-12-09 12:55:00,2024-10-11 18:50:00
4,00360c54a3538e702016303eb89a7c09b0b184505de311...,3,673.0500,2019.15,2024-11-24 01:04:00,2024-10-06 23:39:00
...,...,...,...,...,...,...
11540,ffd4ed11bcb145bd13ba78d9dc07d6ddf557cb134705aa...,1,325.5000,325.50,2024-12-09 20:41:00,2024-12-09 20:41:00
11541,ffdc7158b99eae499b604b662edb5ab16a90373ff0b87a...,1,577.5000,577.50,2024-11-08 14:35:00,2024-11-08 14:35:00
11542,ffe786f8e9251c9304a25b4c2d1992798361dcd79cc2cd...,1,367.5000,367.50,2025-01-16 17:59:00,2025-01-16 17:59:00
11543,fff4cfaf9f880302dbb8e82c859dc459599b53afe2fdab...,1,639.4500,639.45,2024-12-11 23:11:00,2024-12-11 23:11:00


In [10]:
## Recency & Customer Tenure
customer_features["recency_days"] = (
    reference_date - customer_features["last_order_date"]
).dt.days

customer_features["customer_tenure_days"] = (
    customer_features["last_order_date"] - customer_features["first_order_date"]
).dt.days

In [11]:
customer_features

,Customer ID,total_orders,avg_order_value,total_spend,last_order_date,first_order_date,recency_days,customer_tenure_days
0,000285ae83ecf06a92b936d4f5b74342edb0e1940e1f00...,3,453.6000,1360.80,2024-10-01 21:32:00,2024-09-03 21:41:00,122,27
1,001ab5fc3ee158b4d22e106897cee9b355e6eed50f163e...,1,1332.4000,1332.40,2025-01-10 01:38:00,2025-01-10 01:38:00,21,0
2,001fdf2511dd137361424c9c15ea54774476691ceec5fa...,1,1352.4000,1352.40,2024-12-13 22:26:00,2024-12-13 22:26:00,49,0
3,002afbd83626f0c699892fcb07b2ddf3858911482647b6...,4,805.6225,3222.49,2024-12-09 12:55:00,2024-10-11 18:50:00,53,58
4,00360c54a3538e702016303eb89a7c09b0b184505de311...,3,673.0500,2019.15,2024-11-24 01:04:00,2024-10-06 23:39:00,68,48
...,...,...,...,...,...,...,...,...
11540,ffd4ed11bcb145bd13ba78d9dc07d6ddf557cb134705aa...,1,325.5000,325.50,2024-12-09 20:41:00,2024-12-09 20:41:00,53,0
11541,ffdc7158b99eae499b604b662edb5ab16a90373ff0b87a...,1,577.5000,577.50,2024-11-08 14:35:00,2024-11-08 14:35:00,84,0
11542,ffe786f8e9251c9304a25b4c2d1992798361dcd79cc2cd...,1,367.5000,367.50,2025-01-16 17:59:00,2025-01-16 17:59:00,15,0
11543,fff4cfaf9f880302dbb8e82c859dc459599b53afe2fdab...,1,639.4500,639.45,2024-12-11 23:11:00,2024-12-11 23:11:00,51,0


In [20]:
## Frequency Features
def orders_in_last_n_days(df, n_days):
    cutoff = reference_date - pd.Timedelta(days=n_days)
    return (
        df[df["Order Placed At"] >= cutoff]
        .groupby("Customer ID")["Order ID"]
        .count()
    )

customer_features["orders_last_30d"] = customer_features["Customer ID"].map(
    orders_in_last_n_days(df, 30)
)

customer_features["orders_last_60d"] = customer_features["Customer ID"].map(
    orders_in_last_n_days(df, 60)
)

customer_features["orders_last_90d"] = customer_features["Customer ID"].map(
    orders_in_last_n_days(df, 90)
)

customer_features.fillna(0, inplace=True)

customer_features.fillna(0, inplace=True)

In [23]:
customer_features

,Customer ID,total_orders,avg_order_value,total_spend,last_order_date,first_order_date,recency_days,customer_tenure_days,orders_last_30d,orders_last_60d,orders_last_90d
0,000285ae83ecf06a92b936d4f5b74342edb0e1940e1f00...,3,453.6000,1360.80,2024-10-01 21:32:00,2024-09-03 21:41:00,122,27,0.0,0.0,0.0
1,001ab5fc3ee158b4d22e106897cee9b355e6eed50f163e...,1,1332.4000,1332.40,2025-01-10 01:38:00,2025-01-10 01:38:00,21,0,1.0,1.0,1.0
2,001fdf2511dd137361424c9c15ea54774476691ceec5fa...,1,1352.4000,1352.40,2024-12-13 22:26:00,2024-12-13 22:26:00,49,0,0.0,1.0,1.0
3,002afbd83626f0c699892fcb07b2ddf3858911482647b6...,4,805.6225,3222.49,2024-12-09 12:55:00,2024-10-11 18:50:00,53,58,0.0,2.0,3.0
4,00360c54a3538e702016303eb89a7c09b0b184505de311...,3,673.0500,2019.15,2024-11-24 01:04:00,2024-10-06 23:39:00,68,48,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
11540,ffd4ed11bcb145bd13ba78d9dc07d6ddf557cb134705aa...,1,325.5000,325.50,2024-12-09 20:41:00,2024-12-09 20:41:00,53,0,0.0,1.0,1.0
11541,ffdc7158b99eae499b604b662edb5ab16a90373ff0b87a...,1,577.5000,577.50,2024-11-08 14:35:00,2024-11-08 14:35:00,84,0,0.0,0.0,1.0
11542,ffe786f8e9251c9304a25b4c2d1992798361dcd79cc2cd...,1,367.5000,367.50,2025-01-16 17:59:00,2025-01-16 17:59:00,15,0,1.0,1.0,1.0
11543,fff4cfaf9f880302dbb8e82c859dc459599b53afe2fdab...,1,639.4500,639.45,2024-12-11 23:11:00,2024-12-11 23:11:00,51,0,0.0,1.0,1.0


In [24]:
## Discount & Price Sensitivity Features
df["total_discount"] = (
    df["Gold discount"]
    + df["Brand pack discount"]
    + df["Restaurant discount (Promo)"]
    + df["Restaurant discount (Flat offs, Freebies & others)"]
)

discount_features = df.groupby("Customer ID").agg(
    avg_discount=("total_discount", "mean"),
    discount_usage_rate=("total_discount", lambda x: (x > 0).mean())
).reset_index()

customer_features = customer_features.merge(
    discount_features, on="Customer ID", how="left"
)

In [27]:
import re

def parse_distance(x):
    if pd.isna(x):
        return np.nan

    x = str(x).lower().strip()

    # handle cases like "<1" or "<1 km"
    if x.startswith("<"):
        return 0.5  # assume midpoint for "<1 km"

    # extract numeric part
    num = re.findall(r"[\d\.]+", x)
    if not num:
        return np.nan

    value = float(num[0])

    if "m" in x and "km" not in x:
        return value / 1000  # meters to km

    return value  # already in km

df["distance_km"] = df["Distance"].apply(parse_distance)

In [28]:
## Operational Experience Features
ops_features = df.groupby("Customer ID").agg(
    avg_kpt=("KPT duration (minutes)", "mean"),
    avg_rider_wait=("Rider wait time (minutes)", "mean"),
    avg_distance=("distance_km", "mean")
).reset_index()

customer_features = customer_features.merge(
    ops_features, on="Customer ID", how="left"
)

In [29]:
## Churn Label Creation
customer_features["churn"] = np.where(
    customer_features["recency_days"] > 30, 1, 0
)

customer_features["churn"].value_counts(normalize=True) * 100

churn
1    73.962754
0    26.037246
Name: proportion, dtype: float64

In [30]:
## Final Feature Set
final_features = customer_features[[
    "total_orders",
    "avg_order_value",
    "total_spend",
    "recency_days",
    "customer_tenure_days",
    "orders_last_30d",
    "orders_last_60d",
    "orders_last_90d",
    "avg_discount",
    "discount_usage_rate",
    "avg_kpt",
    "avg_rider_wait",
    "avg_distance",
    "churn"
]]

final_features.head()

,total_orders,avg_order_value,total_spend,recency_days,customer_tenure_days,orders_last_30d,orders_last_60d,orders_last_90d,avg_discount,discount_usage_rate,avg_kpt,avg_rider_wait,avg_distance,churn
0,3,453.6000,1360.80,122,27,0.0,0.0,0.0,99.000000,1.000000,16.483333,4.133333,5.0,1
1,1,1332.4000,1332.40,21,0,1.0,1.0,1.0,198.050000,1.000000,14.630000,6.100000,3.0,0
2,1,1352.4000,1352.40,49,0,0.0,1.0,1.0,270.000000,1.000000,29.470000,2.900000,2.0,1
3,4,805.6225,3222.49,53,58,0.0,2.0,3.0,103.740000,1.000000,18.772500,2.750000,5.5,1
4,3,673.0500,2019.15,68,48,0.0,0.0,1.0,37.333333,0.333333,15.866667,3.333333,3.0,1


In [31]:
## Save Processed Dataset
final_features.to_csv("../data/processed/customer_churn_features.csv", index=False)